# The basics

Before we get started with the actual assignment please fill in you name, student ID and group below. Double click this *Markdown* cell to edit the content.


### Name: Bram Otten

### Student ID: 10992456

### Group: F

Also edit the file name to include this information, replacing *LastName*, *FirstName* and *GroupX*. Edit the file name by double clicking it at the top the page. Improperly formated assignments will not be graded, so make sure to do this right away!


# Sudoku solver

Now that we have all the tools setup, we can get started with the actual assignment for this week. You will build a simple [Sudoku](https://en.wikipedia.org/wiki/Sudoku) solver using some of the installed tools, in order to get a little more familiar with them. There are *11 points* in total for these excersises, indicated in square brackets after the title of each section.

## Loading the puzzle [1 pt]

Provided with the assignment are 2 folders, labeled `easy` and `hard`, each containing 3 puzzles of their respective difficulties. You can open the files with any text editor and see they are simply [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) formatted files with 9 rows and 9 columns. Some of the cells are are already filled with numbers `1` to `9`, while others have a `0` value to indicate they are still blank.

A Sudoku puzzle is nothing more than a grid of numbers, so you can very naturally represent the puzzle as a 2-dimensional array. As mentioned in the previous section, *NumPy* is great for handling any multidimensional arrays. In this case you can use the [loadtxt](https://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html) function to convert the CSV file to a *Numpy* array.

Below is some code to get you started. There is the import of *Numpy*, a constant that will come in handy later and the start of your function definition. Finish the function so it returns a *Numpy* array of the provided `filename` argument. The test below the function should then print out the array, so you can see what the puzzle looks like.

In [ ]:
import numpy as np

MAX_VALUE = 9

def load_sudoku(filename):
    file = open(filename, 'r')
    return np.loadtxt(file, delimiter=',', dtype="int")

sudoku = load_sudoku('easy/puzzle1.csv')
print(sudoku)

## Cell indices [1 pt]

Now that we have the puzzle stored as an array, we can easily access a cell at a specific coordinate using the array indices. Some basics are covered [here](https://docs.scipy.org/doc/numpy/user/basics.indexing.html). Most features work exactly the same as for lists, except that for a 2-dimensional array in *NumPy* you would write `array[3, 4]` instead of the traditional `list[3][4]`.

Use the indexing to write the two simple functions for the *Sudoku* grid and test that they work.

In [ ]:
def get_value(grid, x, y):
    
    # Could (aka should) copy-paste some check like this to every f
    if x > MAX_VALUE or x < 0 or y > MAX_VALUE or y < 0:
        return "OUT OF BOUNDS"
    
    return grid[y, x]

def set_value(grid, x, y, val):
    grid[y, x] = val
    
# Test
print(sudoku)
original = get_value(sudoku, 7, 8)
set_value(sudoku, 7, 8, 42)
set_value(sudoku, 7, 8, 42)
print(sudoku)
set_value(sudoku, 7, 8, original)
print(sudoku)

## Rows and columns  [1 pt]

In order to solve a *Sudoku*, you need to be able to find what values are already filled in a cells row or column. Basic slicing of indices works for *Numpy* arrays just as it does for lists. For a complete overview of the options, see [the documentation](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html).

You can use these options to select individual rows or column from the grid. Complete the two functions below and make sure to test them to see if they do what you would expect.

In [ ]:
def get_row(grid, y):
    return grid[y, :]
    
def get_column(grid, x):
    return grid[:, x]
    
# Test
print(sudoku)
print(get_row(sudoku, 2))
print(get_column(sudoku, 6))

## Sudoku squares [2 pts]

Now that you can extract rows and columns, logically the next step is to extract the 3x3 squares. This is a little trickier, but not that hard to do with a short helper function. First write a function that computes the upper and lower bound of the box containing and index. If for example you wanted the upper and lower bound for the index `5`, your function should return `3, 6`, because `5` is in between box-borders `3` and `6`.
    
    | 0 1 2 | 3 4 5 | 6 7 8 |
    0 ..... 3 ..... 6 ..... 9

If you use this function for the x and y position of your cells, you will know exactly what parts of the grid to slice the square from. This will make the actual `get_square` function a lot more straight foward.

Fill in the code for both functions below and test them to see the produce the correct output.

In [ ]:
def square_bounds(ind):
    
    if MAX_VALUE != 9:
        return "NOT IMPLEMENTED THIS GRID SIZE"
    
    if ind in [0, 1, 2]:
        return (0, 3)
    if ind in [3, 4, 5]:
        return (3, 6)
    if ind in [6, 7, 8]:
        return (6, 9)

def get_square(grid, x, y):
    
    # This one is a little confusing because np arrays are
    # used. Could nest some for's and use get_value.
    (npy1, npy2) = square_bounds(x)
    (npx1, npx2) = square_bounds(y)
    return grid[npx1:npx2, npy1:npy2]

# Test
print(sudoku)
print(square_bounds(2))
print(get_square(sudoku, 7, 0))

## Possible cell values [2 pts]

Now we can access the values already filled in the rows, columns and squares. However, in order to solve the sudoku we need the opposite information; which values can still be used for an open cell. For this write a function `not_used`, which takes a set values and returns which of the 9 numbers have not been used yet. One possible solution might be to use the *Numpy* function [setdiff1d](https://docs.scipy.org/doc/numpy/reference/generated/numpy.setdiff1d.html), but there are plenty of other valid solutions.

With all these functions written, finding out what valid values can be used to fill a cell is actually pretty easy. You can simply use `get_row`, `get_column` and `get_square` and figure out what values are `not_used` for each of these. Then all that remains is find the unused values that occur in all 3 of the unused value sets, meaning they are actually valid options for that specific cell. Again, a possible solution might be to use another *NumPy* set function, [intersect1d](https://docs.scipy.org/doc/numpy/reference/generated/numpy.intersect1d.html), but feel free to write your own solution. Fill in the code for the function `possible_values` below and test it for some coordinates.

In [ ]:
def not_used(vals):
    numbers = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
    values = np.array(vals)
    return np.setdiff1d(numbers, values)

def possible_values(grid, x, y):
    
    freeRow = not_used(get_row(grid, y))
    freeColumn = not_used(get_column(grid, x))
    freeSquare = not_used(get_square(grid, x, y))
    
    return np.intersect1d(np.intersect1d(freeRow, freeColumn), freeSquare)

# Test
print(not_used([2, 3, 4]))
print(not_used(np.array([4, 2])))
print(sudoku)
print(possible_values(sudoku, 7, 1))

## Simple solutions [2 pts]

Now that we can find the possible values of a cell, solving the *Sudoku* is just repeating that process until the grid is completely solved! First write a short function to detect a square has been complete solved, called `is_solved`. Remember that cells that have not been filled yet have the value `0`.

Now for a simple solution you can just loop through the grid coordinates repeatedly, finding cells which have not been filled yet. Then, using the `possible_values` function you can determine what the options are for that cell. If there is only **1** possible option for an unfilled cell, you can fill that cell with that option. Repeat this process until every cell in the grid is filled.

In [ ]:
def is_solved(grid):
    return not 0 in grid

def set_global(grid):
    global globalState
    globalState = grid

def solve_simple(grid, newHere = True):
    
    if newHere:
        set_global(False)
        # Could check if the given grid is even valid.
        # Theoretically, it can only not be just 
        # after reading a file.
        # for in for etc., check if actual value
        # is in possible values
                 
    # Get next problem (done if there are none)
    (yList, xList) = np.where(grid == 0)
    if xList.size == 0:
        set_global(grid)
        return globalState
    
    # Fill in the first problem that can only be solved
    # one way. Then do the whole thing again
    for index in range(yList.size):
        x = xList[index]
        y = yList[index]
        pos = possible_values(grid, x, y)
        if pos.size == 0:
            return False
        elif pos.size == 1:
            set_value(grid, x, y, pos[0])
            solve_simple(grid, False)
            break
    
    return globalState

# Test
sudoku = load_sudoku('easy/puzzle3.csv')
print(sudoku, is_solved(sudoku))
print("Solving...")
print(solve_simple(sudoku))

## Diving in depth first [2 pts]

As you may have found out, this simple solution method works fine for the `easy` puzzles, but fails to find solutions for the `hard` puzzles. This means that when trying to solve those puzzle, the grid might not have obvious cells to fill (i.e. all unfilled cells have multiple possible values). To solve them, you could try to search through all possible solutions to see if you can find the correct one.

This kind of tree seach problem should conjure images of your *Prolog* days, and indeed can be solved using [Depth First Search](https://en.wikipedia.org/wiki/Depth-first_search). You can write recursive functions in *Python* as well, just make sure to `return` the correct result to the function that made the recursive call. However, these are all just suggestions. Feel free to use [Breath First Search](https://en.wikipedia.org/wiki/Breadth-first_search) instead and use an iterative solution instead of recursion. Fill in the function below and test it on both `easy` (should give the same answer as `solve_simpe`) and `hard` problems.

In [ ]:
def solve_complete(grid, newHere = True):
    
    if newHere:
        set_global(False)
    
    # Get next problem (done if there are none)
    (yList, xList) = np.where(grid == 0)
    if xList.size < 1:
        set_global(grid)
        return globalState
    x = xList[0]
    y = yList[0]
    
    pos = possible_values(grid, x, y)
    if pos.size == 0:
        return False
    
    # Try implementing, on a copied board
    # because this may be a dead end
    for value in pos:
        gridCopy = np.copy(grid)
        set_value(gridCopy, x, y, value)
        solve_complete(gridCopy, False)
    
    return globalState
    
puzzle = load_sudoku('hard/puzzle6.csv')
print(puzzle)
print("Solving...")
print(solve_complete(puzzle))